Web Scraping Request

In [1]:
import json
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
headers = {
    'accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Encoding' : 'gzip, deflate, br, zstd',
    'Accept-Language' : 'en-US,en;q=0.9',
    'Cache-Control' : 'max-age=0',
    'Cookie' : 'zguid=24|%24cd571bae-3bb2-4c71-8af7-219330e0144e; zgsession=1|e7b011f9-fbfc-4d97-be39-b94a5685449c; _ga=GA1.2.552083832.1712946886; _gid=GA1.2.1948147929.1712946886; zjs_anonymous_id=%22cd571bae-3bb2-4c71-8af7-219330e0144e%22; zjs_user_id=null; zg_anonymous_id=%2217c8a19d-84d6-4ea3-9535-cc338b2b92ec%22; pxcts=566ed757-f8fb-11ee-a5b7-f30da94b57c3; _pxvid=566ebeae-f8fb-11ee-a5b6-835c77c5abe1; _gcl_au=1.1.247842125.1712946888; _scid=d827d90d-748b-4aab-9632-d0718f9dedd4; __pdst=682b313b04b640dc9b66e7f288b218ba; _fbp=fb.1.1712946888259.1929888891; _pin_unauth=dWlkPVptVXlPR1V4TkRVdE9URmtNaTAwWkRJNUxUa3haREV0TW1OaU5UQTRZV00yTW1FMw; _clck=fv6oo5%7C2%7Cfkv%7C0%7C1563; DoubleClickSession=true; _sctr=1%7C1712894400000; _hp2_id.1215457233=%7B%22userId%22%3A%22818363225522989%22%2C%22pageviewId%22%3A%221923431096912517%22%2C%22sessionId%22%3A%226855505058176367%22%2C%22identity%22%3Anull%2C%22trackerVersion%22%3A%224.0%22%7D; JSESSIONID=B9A26C4342CD0EB188BAA2E7F76AF84C; AWSALB=+FKJa5FoMBZZrRjkdJqX97J43R4FMwYqFIUQGWPqGkSCuQcmIlCD8wMV61KEgJ5qB8LzEklzgfdOETnRDre3QceH1H/voUHoQ6iKTSmTUBM+y9zUkYOfN6Mc8AqE; AWSALBCORS=+FKJa5FoMBZZrRjkdJqX97J43R4FMwYqFIUQGWPqGkSCuQcmIlCD8wMV61KEgJ5qB8LzEklzgfdOETnRDre3QceH1H/voUHoQ6iKTSmTUBM+y9zUkYOfN6Mc8AqE; _px3=3f403d85d0c55ad8f0f3f341831e84bbf41cffb91762c45c55da4e1bd1111625:nXYDsSG70B/tVtxeslPIdlNUdf2m03yD0V+VIr/33HhWDPnwtTQnVgU0j84EjOcn1vXcxFKGofxR60OmYudBoA==:1000:XxH29ZFWOmxcl2D8jy/v+ZQngvvB69S2ApIrSurDkUijL8i9u1nxGXCi51xLydjNg4gykDtgOCwVPNvx5DIOw8nhtJzh5OiQx+jnDK+I87pYf0+nbtS5oA8vXifHokTewTlthnDI7KgzUsXhoqw06/MJ+WRtp374+fGaq69YcMjztUjIELXRRpf/GeYQAuFJZEhhwUqyB0MwYnXYyHdnUQYNiF9Lj552qZZl/ghfiag=; _rdt_uuid=1712946887833.20a354fd-d0d4-4367-bd61-9bf91c306468; _scid_r=d827d90d-748b-4aab-9632-d0718f9dedd4; _uetsid=575c5ae0f8fb11ee8621d166739b4420; _uetvid=7d16be50867e11ee8f095f09ed602a72; tfpsi=aac89f84-dd48-4826-8fbd-bf1e32eacae0; __gads=ID=20410234d7cd9782:T=1712946972:RT=1712952114:S=ALNI_MbL0ieM0vH_p88bp_ukjOwVzncnSg; __gpi=UID=00000ddd8b8f31b2:T=1712946972:RT=1712952114:S=ALNI_MbijLE2TPT6IAb1P98T3ipR75X2Fw; __eoi=ID=af4af501323fc4cf:T=1712946972:RT=1712952114:S=AA-AfjZFMEYjgTSpLsjZPnG9w1C6; search=6|1715544124480%7Crect%3D41.19614368179375%2C-74.75680292578124%2C36.342699121024545%2C-79.71713007421874%26rid%3D27%26disp%3Dmap%26mdm%3Dauto%26p%3D1%26z%3D1%26listPriceActive%3D1%26fs%3D1%26fr%3D0%26mmm%3D0%26rs%3D0%26ah%3D0%26singlestory%3D0%26housing-connector%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%26featuredMultiFamilyBuilding%3D0%26student-housing%3D0%26income-restricted-housing%3D0%26military-housing%3D0%26disabled-housing%3D0%26senior-housing%3D0%26commuteMode%3Ddriving%26commuteTimeOfDay%3Dnow%09%0927%09%7B%22isList%22%3Atrue%2C%22isMap%22%3Afalse%7D%09%09%09%09%09; x-amz-continuous-deployment-state=AYABeDWGgRf7kP8Zjz%2F+u5KMLywAPgACAAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADI4xNExlWXzY3Oj8wQAwaq9Th1lO8FrTJ%2FukLnWczjb2odiW9QaYXqxInU3LGGyWfC+CIXAYQdbvv+XDqs6yAgAAAAAMAAQAAAAAAAAAAAAAAAAAAEb5H9Oe3ikQbHigL%2Frf3hT%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAxLx63OONdpAvK63vBpgTFEU9fRQhmEsgHxYR54U9fRQhmEsgHxYR54U9fRQhmEsgHxYR54U9fRQhmEsgHxYR54; _clsk=zz4ke6%7C1712952125690%7C2%7C0%7Cj.clarity.ms%2Fcollect',
    'Sec-Fetch-Dest' : 'document',
    'Sec-Fetch-Mode' : 'navigate',
    'Sec-Fetch-Site' : 'same-origin',
    'Sec-Fetch-User' : '?1',
    'Upgrade-Insecure-Requests' : '1',
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
         }

In [13]:
url = 'https://www.zillow.com/orange-county-ny/'

In [14]:
params = {'searchQueryState' : '{"pagination":{},"isMapVisible":false,"mapBounds":{"west":-79.71713007421874,"east":-74.75680292578124,"south":36.35818393143443,"north":41.18167524595343},"usersSearchTerm":"MD","regionSelection":[{"regionId":27,"regionType":2}],"filterState":{"sort":{"value":"globalrelevanceex"},"con":{"value":false},"land":{"value":false},"ah":{"value":true},"apa":{"value":false},"manu":{"value":false},"apco":{"value":false},"hoa":{"min":null,"max":200},"basf":{"value":true},"sch":{"value":true},"schr":{"value":false},"schc":{"value":false},"schb":{"min":7},"schu":{"value":false}},"isListVisible":true,"mapZoom":8}'
         }

In [15]:
response = requests.get(url = url, headers=headers, params=params)
response

<Response [200]>

In [24]:
content = BeautifulSoup(response.text, 'lxml')
card_set = content.find('ul', class_ = 'List-c11n-8-100-7__sc-1smrmqp-0 StyledSearchListWrapper-srp-8-100-7__sc-1ieen0c-0 fybckg kescig photo-cards photo-cards_extra-attribution')
#house_card = card_set.find_all('li', class_ = 'ListItem-c11n-8-100-7__sc-13rwu5a-0 StyledListCardWrapper-srp-8-100-7__sc-wtsrtn-0 eSVUTI egMeDW')
# house_price = card_set.find_all('div', class_ = 'PropertyCardWrapper__StyledPriceGridContainer-srp-8-100-7__sc-16e8gqd-0 gHBMRX')
# rooms = card_set.find_all('div', class_ = 'StyledPropertyCardDataArea-c11n-8-100-7__sc-10i1r6-0 eObjGE')
# address = card_set.find_all('address', {'data-test': 'property-card-addr'})
# test = card_set.find_all('script', {'type': 'application/ld+json'})

In [25]:
response.txt

AttributeError: 'Response' object has no attribute 'txt'

In [27]:
try:
    # Send a GET request to the URL
    response = requests.get(url)
    response.raise_for_status()  # Check for any HTTP errors

    # Parse HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract data using BeautifulSoup
   
    prices = soup.find_all(class_="list-card-price")

    # Print the prices
    for price in prices:
        print(price.text)

except Exception as e:
    print("An error occurred:", e)

An error occurred: 403 Client Error: Forbidden for url: https://www.zillow.com/orange-county-ny/
